<a href="https://colab.research.google.com/github/MaliheDahmardeh/state-house-price-analysis-SM/blob/main/state-house-price-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [66]:
!ls

sample_data  State_house_prices_ts_usa.csv


In [67]:
df_house_price = pd.read_csv('State_house_prices_ts_usa.csv')
df_house_price

,Date;RegionName;Price_1bedroom;Price_2bedroom;Price_3bedroom;Price_4bedroom;Price_5BedroomOrMore;Price_SingleFamilyResidence
0,30/04/96;Alabama;61500;48900;78200;146500;2063...
1,30/04/96;Arizona;59200;86400;96100;128400;1905...
2,30/04/96;Arkansas;53000;54500;76800;135100;186...
3,30/04/96;California;93700;123400;150900;196100...
4,30/04/96;Colorado;77800;97500;129000;176100;21...
...,...
13207,31/12/17;Virginia;200600;170300;221200;367800;...
13208,31/12/17;Washington;275700;289600;347600;46590...
13209,31/12/17;WestVirginia;;;113900;155500;181400;1...
13210,31/12/17;Wisconsin;118200;128100;169600;222600...


In [68]:
df_house_price = df_house_price['Date;RegionName;Price_1bedroom;Price_2bedroom;Price_3bedroom;Price_4bedroom;Price_5BedroomOrMore;Price_SingleFamilyResidence'].str.split(';', expand=True)
df_house_price.rename(columns = { 0:'Date', 1:'RegionName',
                                  2:'Price_1bedroom', 3:'Price_2bedroom', 4:'Price_3bedroom', 5:'Price_4bedroom', 
                                  6:'Price_5BedroomOrMore', 7:'Price_SingleFamilyResidence'}, inplace = True)
df_house_price

,Date,RegionName,Price_1bedroom,Price_2bedroom,Price_3bedroom,Price_4bedroom,Price_5BedroomOrMore,Price_SingleFamilyResidence
0,30/04/96,Alabama,61500,48900,78200,146500,206300,79000
1,30/04/96,Arizona,59200,86400,96100,128400,190500,107500
2,30/04/96,Arkansas,53000,54500,76800,135100,186000,64500
3,30/04/96,California,93700,123400,150900,196100,265300,162000
4,30/04/96,Colorado,77800,97500,129000,176100,212900,133600
...,...,...,...,...,...,...,...,...
13207,31/12/17,Virginia,200600,170300,221200,367800,531800,237300
13208,31/12/17,Washington,275700,289600,347600,465900,580300,352800
13209,31/12/17,WestVirginia,,,113900,155500,181400,103300
13210,31/12/17,Wisconsin,118200,128100,169600,222600,261200,170200


#cleaning

In [69]:
df_house_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13212 entries, 0 to 13211
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Date                         13212 non-null  object
 1   RegionName                   13212 non-null  object
 2   Price_1bedroom               13212 non-null  object
 3   Price_2bedroom               13212 non-null  object
 4   Price_3bedroom               13212 non-null  object
 5   Price_4bedroom               13212 non-null  object
 6   Price_5BedroomOrMore         13212 non-null  object
 7   Price_SingleFamilyResidence  13212 non-null  object
dtypes: object(8)
memory usage: 825.9+ KB


In [70]:
df_house_price.describe()

,Date,RegionName,Price_1bedroom,Price_2bedroom,Price_3bedroom,Price_4bedroom,Price_5BedroomOrMore,Price_SingleFamilyResidence
count,13212,13212,13212,13212,13212,13212,13212,13212
unique,261,52,2233,2541,2846,3524,4437,3115
top,31/08/12,Alabama,,,,,,
freq,52,261,2607,1467,425,853,1398,774


In [72]:
df_house_price.duplicated().sum()   

0